# **LangChain Memory**

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

We call this ability to store information about past interactions "memory". LangChain provides a lot of utilities for adding memory to a system. These utilities can be used by themselves or incorporated seamlessly into a chain.

## **Building memory into a system**
The two core design decisions in any memory system are:
- How state is stored
- How state is queried

## **What's covered?**
- ConversationBufferMemory
- End-to-end Example: Conversational AI Bot
- Saving and Loading a Chat History
- ChatMessageHistory [Beta]
- ConversationBufferWindowMemory [Beta]
- ConversationSummaryMemory [Beta]

## **ConversationBufferMemory**

Let's take a look at how to use ConversationBufferMemory in chains. ConversationBufferMemory is an extremely simple form of memory that just keeps a list of chat messages in a buffer and passes those into the prompt template.

This memory type can be connected to a conversation. It allows for storing messages and then extracts the messages in a variable.

In [1]:
# Importing ConversationBufferMemory
from langchain.memory import ConversationBufferMemory

# Set up an instance of Conversation buffer memory
memory = ConversationBufferMemory()

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Loading Memory

memory.load_memory_variables({})

{'history': ''}

In [3]:
# Loading memory buffer with history

memory.buffer

''

### **Renaming the memory variable and Adding Messages**

In [4]:
memory = ConversationBufferMemory(memory_key="chat_history")

memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [5]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

In [6]:
memory.buffer

"Human: hi!\nAI: what's up?"

### **By default, memory is returned as a single string. In order to return as a list of messages, you can set `return_messages=True`**

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [8]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi!'),
  AIMessage(content="what's up?")]}

In [9]:
memory.buffer

[HumanMessage(content='hi!'), AIMessage(content="what's up?")]

## **End-to-end Example: Conversational AI Bot**

<img src="images/memory.png">

### **Steps:**
1. Import Chat Model and Configure the API Key
2. Create Chat Template
3. Initialize the Memory
4. Create a Output Parser
5. Build a Chain
6. Invoke the chain with human_input and chat_history
7. Saving to memory
8. Run Step 6 and 7 in a loop

In [10]:
# Step 1 - Import Chat Model and Configure the API Key

from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [11]:
# Step 2 - Create Chat Template

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        # The persistent system prompt
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),
        # Creating a chat_history placeholder
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  
        # Human Prompt
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

In [12]:
# Step 3 - Initialize the Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [13]:
# Step 4 - Create a Output Parser

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

#### **RunnablePassthrough:** RunnablePassthrough on its own allows you to pass inputs unchanged.

In [14]:
# Step 5 - Build a Chain (Another way)
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Define a function to load the message from memory
def get_messages_from_memory(human_input):
    return memory.load_memory_variables(human_input)['chat_history']

# Define a chain
chain = RunnablePassthrough.assign(
            chat_history=RunnableLambda(get_messages_from_memory)
        ) | chat_prompt_template | chat_model | output_parser

In [15]:
# Step 6 - Invoke the chain with human_input and chat_history

query = {"human_input": "Hi, How are you?"}

response = chain.invoke(query)

response

"Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?"

In [16]:
memory.buffer

[]

In [17]:
# Step 7 - Saving to memory

memory.save_context(query, {"output" : response})

memory.buffer

[HumanMessage(content='Hi, How are you?'),
 AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?")]

In [20]:
# Step 8 - Run Step 6 and 7 in a loop

while True:
    query = {"human_input" : input('Enter your input: ')}
    print(f"*User: {query['human_input']}")
    if query["human_input"] in ['bye', 'quit', 'exit']:
        break
    response = chain.invoke(query)
    print(f"*AI: {response}")

    memory.save_context(query, {"output" : response})

Enter your input:  *User:   My name is Kanav. Can you tell me what kind of tasks can you solve for me?


*User: *User:   My name is Kanav. Can you tell me what kind of tasks can you solve for me?
*AI: Hello Kanav! I can help answer questions, provide information, offer suggestions, assist with problem-solving, and engage in conversation on a variety of topics. Feel free to ask me anything you'd like help with!


Enter your input:  That's good to know.


*User: That's good to know.
*AI: I'm glad you think so! If you have any questions or need assistance with anything, feel free to ask. I'm here to help!


Enter your input:  What is my name?


*User: What is my name?
*AI: Your name is Kanav.


Enter your input:  bye


*User: bye


In [21]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi, How are you?'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='*User:   My name is Kanav. Can you tell me what kind of tasks can you solve for me?'),
  AIMessage(content="Hello Kanav! I can help answer questions, provide information, offer suggestions, assist with problem-solving, and engage in conversation on a variety of topics. Feel free to ask me anything you'd like help with!"),
  HumanMessage(content="That's good to know."),
  AIMessage(content="I'm glad you think so! If you have any questions or need assistance with anything, feel free to ask. I'm here to help!"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Kanav.')]}

## **Saving a Chat History**

**Let's now learn to save this history on the disk so that whenever we can load the history whenever we chat with our assistant.**

In [22]:
import pickle

chat_history = pickle.dumps(memory)

with open("chats_data/conversation_memory.pkl", "wb") as f:
    f.write(chat_history)

## **Loading a Chat History**

In [23]:
chat_history_loaded = pickle.load(open("chats_data/conversation_memory.pkl", "rb"))

print(chat_history_loaded)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, How are you?'), AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"), HumanMessage(content='*User:   My name is Kanav. Can you tell me what kind of tasks can you solve for me?'), AIMessage(content="Hello Kanav! I can help answer questions, provide information, offer suggestions, assist with problem-solving, and engage in conversation on a variety of topics. Feel free to ask me anything you'd like help with!"), HumanMessage(content="That's good to know."), AIMessage(content="I'm glad you think so! If you have any questions or need assistance with anything, feel free to ask. I'm here to help!"), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Kanav.')]) return_messages=True memory_key='chat_history'


In [24]:
chat_history_loaded.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi, How are you?'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='*User:   My name is Kanav. Can you tell me what kind of tasks can you solve for me?'),
  AIMessage(content="Hello Kanav! I can help answer questions, provide information, offer suggestions, assist with problem-solving, and engage in conversation on a variety of topics. Feel free to ask me anything you'd like help with!"),
  HumanMessage(content="That's good to know."),
  AIMessage(content="I'm glad you think so! If you have any questions or need assistance with anything, feel free to ask. I'm here to help!"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Kanav.')]}

## **ConversationBufferWindowMemory [Beta]**

With this we will have an ability to call back just a window of conversation.

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [18]:
from langchain_openai import ChatOpenAI

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [19]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        # The persistent system prompt
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),
        # Creating a chat_history placeholder
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  
        # Human Prompt
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

In [20]:
from langchain.memory import ConversationBufferWindowMemory

# Set up an instance of Conversation buffer window memory
window_memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=2)
# here, k most recent interactions will be refered to answer the user queries

In [21]:
window_memory.load_memory_variables({})

{'chat_history': []}

In [45]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Define a function to load the message from memory
def get_messages_from_memory(human_input):
    return window_memory.load_memory_variables(human_input)['chat_history']

# Define a chain
chain = RunnablePassthrough.assign(
            chat_history=RunnableLambda(get_messages_from_memory)
        ) | chat_prompt_template | chat_model

In [23]:
query = {"human_input" : "What are the top 3 states of India on the basis of population?"}

response = chain.invoke(query)

window_memory.save_context(query, {"output" : response.content})

print(response.content)

The top 3 most populous states in India are Uttar Pradesh, Maharashtra, and Bihar.


In [24]:
window_memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What are the top 3 states of India on the basis of population?'),
  AIMessage(content='The top 3 most populous states in India are Uttar Pradesh, Maharashtra, and Bihar.')]}

In [25]:
query = {"human_input" : "What are the next 2 states?"}

response = chain.invoke(query)

window_memory.save_context(query, {"output" : response.content})

print(response.content)

The next two most populous states in India are West Bengal and Madhya Pradesh.


In [26]:
window_memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What are the top 3 states of India on the basis of population?'),
  AIMessage(content='The top 3 most populous states in India are Uttar Pradesh, Maharashtra, and Bihar.'),
  HumanMessage(content='What are the next 2 states?'),
  AIMessage(content='The next two most populous states in India are West Bengal and Madhya Pradesh.')]}

## **ChatMessageHistory**

`ChatMessageHistory` allows us to store separate conversation histories per user or session which is often done by the real-time chatbots. `session_id` is used to distinguish between separate conversations.

In order to use it, we can use a `get_session_history` function which take `session_id` and returns a message history object.

In [46]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
history_store = {}

